### 01 - 测试EC2 Bedrock权限

In [1]:
import json
import boto3

default_region="us-east-1"
model_id="anthropic.claude-3-sonnet-20240229-v1:0"
#model_id="anthropic.claude-3-haiku-20240307-v1:0"

def invoke_claude_3_with_text(prompt):

        # Initialize the Amazon Bedrock runtime client
        client = boto3.client(
            service_name="bedrock-runtime", region_name=default_region
        )

        # Invoke Claude 3 with the text prompt
        try:
            response = client.invoke_model(
                modelId=model_id,
                body=json.dumps(
                    {
                        "anthropic_version": "bedrock-2023-05-31",
                        "max_tokens": 1024,
                        "messages": [
                            {
                                "role": "user",
                                "content": [{"type": "text", "text": prompt}],
                            }
                        ],
                    }
                ),
            )

            # Process and print the response
            result = json.loads(response.get("body").read())
            input_tokens = result["usage"]["input_tokens"]
            output_tokens = result["usage"]["output_tokens"]
            output_list = result.get("content", [])

            print("Invocation details:")
            print(f"- The input length is {input_tokens} tokens.")
            print(f"- The output length is {output_tokens} tokens.")

            print(f"- The model returned {len(output_list)} response(s):")
            for output in output_list:
                print(output["text"])

            return result

        except ClientError as err:
            logger.error(
                "Couldn't invoke Claude 3 Sonnet. Here's why: %s: %s",
                err.response["Error"]["Code"],
                err.response["Error"]["Message"],
            )
            raise


response_body = invoke_claude_3_with_text("Who are you ?")
#print(response_body)

Invocation details:
- The input length is 11 tokens.
- The output length is 96 tokens.
- The model returned 1 response(s):
I am an artificial intelligence called Claude, created by a company called Anthropic. I'm an advanced language model trained to assist humans with a wide variety of tasks through natural conversations. I don't have a physical form, but I can engage in dialogue, answer questions, help with analysis and writing, and much more. I have a strong ethical foundation and I'm committed to being helpful while avoiding harm. Please let me know if you have any other questions!


### 02 Prompt 实现Function Calling (未调用Tools)
通过Prompt 让Claude3返回正确的调用函数以及参数(文本)

In [3]:

import boto3
import os
import json

#set different city name 
my_question= {
    "role": "user",
    "content": "Get the current temperature in Paris, France"
}

tool = """
<tool_description>
    <tool_name>get_temperature</tool_name>
    <description>
        Function for finding the current temperature in a location.
    </description>
    <parameters>
        <parameter>
            <name>location</name>
            <type>str</type>
            <description>Location to get the temp of</description>
        </parameter>
    </parameters>
</tool_description>
"""

system_prompt = f"""
You have access to an additional set of one or more functions you can use to
answer the user's question.

You may call them like this:
<function_calls>
    <invoke>
        <tool_name>$TOOL_NAME</tool_name>
        <parameters>
            <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
            ...
        </parameters>
    </invoke>
</function_calls>

Here are the tools available:
<tools>{tool}</tools>
Your final answer should be of the form 'The current temperature in <location> is <temp> degrees'
"""


bedrock = boto3.client(service_name='bedrock-runtime',region_name="us-east-1")
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
body = json.dumps({
        "anthropic_version":"bedrock-2023-05-31",
        "system": system_prompt,
        "messages": [my_question],
        "stop_sequences":["\n\nHuman:", "</function_calls>"],
        "max_tokens": 2048,
        "temperature": 0})
accept = 'application/json'
contentType = 'application/json'

#bedrock api call with prompt
partial_completion = bedrock.invoke_model(
body=body,
modelId=modelId,
accept=accept,
contentType=contentType
)
response_body = json.loads(partial_completion.get('body').read())

# print(response_body.get('content')["text"])
for content in response_body.get('content'):
    if content.get("type")=="text":
        print(content["text"])
partial_completion= response_body.get('completion')

Okay, let me call the get_temperature tool to find the current temperature in Paris, France.

<function_calls>
    <invoke>
        <tool_name>get_temperature</tool_name>
        <parameters>
            <location>Paris, France</location>
        </parameters>
    </invoke>



### 03 Prompt 实现Function Calling 调用多个Tools

tools.py 工具类,包括具体的API函数，以及API描述


In [4]:
!pygmentize tools.py

import requests

# To add a tool to be used by Claude in main_demo.py,
# create your tool in python as shown below and then create
# a new string variable describing the tool spec. Copy the XML formatting
# that is shown in the below example.
#
# Once you have created your tool and your spec, add the spec variable to the
# list_of_tools_specs list.

def get_weather(latitude: str, longitude: str):
  url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
  response = requests.get(url)
  return response.json()

get_weather_description = """
<tool_description>
<tool_name>get_weather</tool_name>
<description>
Returns weather data for a given latitude and longitude. </description>
<parameters>
<parameter>
<name>latitude</name>
<type>string</type>
<description>The latitude coordinate as a string</description>
</parameter> <parameter>
<name>longitude</name>
<type>string</type>
<description>The longitude coordinate as a string</description

In [5]:
import sys
from defusedxml import ElementTree
from collections import defaultdict
import os
from typing import Any
import tools
import boto3
import json
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

def create_prompt(tools_string, user_input):
    prompt_template = f"""
In this environment you have access to a set of tools you can use to answer the user's question. Your answer must be english.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
{tools_string}
</tools>

Human:
{user_input}


Assistant:
"""
    return prompt_template

def add_tools():
    tools_string = ""
    for tool_spec in tools.list_of_tools_specs:
        tools_string += tool_spec
    return tools_string

def call_function(tool_name, parameters):
    func = getattr(tools, tool_name)
    print(tool_name,parameters,func)
    output = func(**parameters)
    return output

def format_result(tool_name, output):
    return f"""
<function_results>
<result>
<tool_name>{tool_name}</tool_name>
<stdout>
{output}
</stdout>
</result>
</function_results>
"""

def etree_to_dict(t) -> dict[str, Any]:
    d = {t.tag: {}}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k: v[0] if len(v) == 1 else v for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(("@" + k, v) for k, v in t.attrib.items())
    if t.text and t.text.strip():
        if children or t.attrib:
            d[t.tag]["#text"] = t.text
        else:
            d[t.tag] = t.text
    return d


system_prompt = f"""
You have access to an additional set of one or more functions you can use to 
answer the user's question.

You may call them like this:
<function_calls>
    <invoke>
        <tool_name>$TOOL_NAME</tool_name>
        <parameters>
            <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
            ...
        </parameters>
    </invoke>
</function_calls>

Here are the tools available:
<tools>......</tools>
Your final answer should be of the form 'The current temperature in <location> is <temp> degrees, windspeed is km/h' 
Your final answer should use Chinese
"""


def run_loop(prompt):
    # Start function calling loop
    while True:
    # initialize variables to make bedrock api call
        bedrock = boto3.client(service_name='bedrock-runtime',region_name="us-east-1")
        modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
       

        body = json.dumps({
        "anthropic_version":"bedrock-2023-05-31",
        "system": system_prompt,
        "messages": [{'role': 'user', 'content': prompt}],
        "stop_sequences":["\n\nHuman:", "</function_calls>"],
        "max_tokens": 2048,
        "temperature": 0})

        accept = 'application/json'
        contentType = 'application/json'
        # bedrock api call with prompt
        partial_completion = bedrock.invoke_model(
        body=body, 
        modelId=modelId, 
        accept=accept, 
        contentType=contentType
        )
   
        response_body = json.loads(partial_completion.get('body').read())

        for content in response_body.get('content'):
            if content.get("type")=="text":
                print(content["text"])
                partial_completion=content["text"]
        stop_reason=response_body.get('stop_reason')
        stop_seq = partial_completion.rstrip().endswith("</invoke>")

        print(response_body)
        
        # Get a completion from Claude

        # Append the completion to the end of the prommpt
        prompt += partial_completion
        if stop_reason == 'stop_sequence' and stop_seq:
            # If Claude made a function call
            print(partial_completion)
            start_index = partial_completion.find("<function_calls>")
            if start_index != -1:
                # Extract the XML Claude outputted (invoking the function)
                extracted_text = partial_completion[start_index+16:]

                # Parse the XML find the tool name and the parameters that we need to pass to the tool
                print(f"{extracted_text=}")
                xml = ElementTree.fromstring(extracted_text)
                tool_name_element = xml.find("tool_name")
                if tool_name_element is None:
                    print("Unable to parse function call, invalid XML or missing 'tool_name' tag")
                    break
                tool_name_from_xml = tool_name_element.text.strip()
                parameters_xml = xml.find("parameters")
                if parameters_xml is None:
                    print("Unable to parse function call, invalid XML or missing 'parameters' tag")
                    break
                param_dict = etree_to_dict(parameters_xml)
                parameters = param_dict["parameters"]

                # Call the tool we defined in tools.py
                output = call_function(tool_name_from_xml, parameters)

                # Add the stop sequence back to the prompt
                prompt += "</function_calls>"
                print("</function_calls>")

                # Add the result from calling the tool back to the prompt
                function_result = format_result(tool_name_from_xml, output)
                print(function_result)
                prompt += function_result
        else:
            # If Claude did not make a function call
            # outputted answer
            print("\n\nFinal Answer:",partial_completion)
            break

user_input = "Can you check the weather for me in Paris, France?"
user_input = "你能帮我查询一下英国伦敦的天气吗?"

tools_string = add_tools()
prompt = create_prompt(tools_string, user_input)
run_loop(prompt)





<function_calls>
<invoke>
<tool_name>get_lat_long</tool_name>
<parameters>
<place>London, United Kingdom</place>
</parameters>
</invoke>

{'id': 'msg_bdrk_01Avctkqok2t4PZsZN84UE1r', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-sonnet-20240229', 'stop_sequence': '</function_calls>', 'usage': {'input_tokens': 556, 'output_tokens': 54}, 'content': [{'type': 'text', 'text': '<function_calls>\n<invoke>\n<tool_name>get_lat_long</tool_name>\n<parameters>\n<place>London, United Kingdom</place>\n</parameters>\n</invoke>\n'}], 'stop_reason': 'stop_sequence'}
<function_calls>
<invoke>
<tool_name>get_lat_long</tool_name>
<parameters>
<place>London, United Kingdom</place>
</parameters>
</invoke>

extracted_text='\n<invoke>\n<tool_name>get_lat_long</tool_name>\n<parameters>\n<place>London, United Kingdom</place>\n</parameters>\n</invoke>\n'
get_lat_long {'place': 'London, United Kingdom'} <function get_lat_long at 0x7f077b81d940>
b'[{"place_id":273845995,"licence":"Data \xc2\xa9 OpenSt

### Native Function Calling

https://docs.aws.amazon.com/zh_cn/bedrock/latest/userguide/tool-use.html#tool-use-examples

In [1]:
!pygmentize native_fc.py

# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
"""
Shows how to use tools with the Conversation API and the Claude3 model.
"""

import logging
import boto3


from botocore.exceptions import ClientError


class StationNotFoundError(Exception):
    """Raised when a radio station isn't found."""
    pass


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


def get_top_song(call_sign):
    """Returns the most popular song for the requested station.
    Args:
        call_sign (str): The call sign for the station for which you want
        the most popular song.

    Returns:
        response (json): The most popular song and artist.
    """

    song = ""
    artist = ""
    if call_sign == 'WZPZ':
        song = "Elemental Hotel"
        artist = "8 Storey Hike"

    else:
        raise StationNotFoundError(f"Station {call_sign} not found.")

    return song, artist


def generate_text(bedrock_client, 

In [2]:
!python native_fc.py

INFO:botocore.credentials:Found credentials from IAM Role: NativeFunctionCalling-bedrockInferenceRole-QEw2wb6Fmnf7
Question: What is the most popular song on WZPZ?
INFO:__main__:Generating text with model anthropic.claude-3-haiku-20240307-v1:0
INFO:__main__:Requesting tool top_song. Request: tooluse_VNMgQgieQRS3GxWI_ZO58w
The most popular song on radio station WZPZ is "Elemental Hotel" by the artist 8 Storey Hike.
Finished generating text with model anthropic.claude-3-haiku-20240307-v1:0.
